In [1]:
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import matplotlib.image as mpimg
import os
import xml.etree.ElementTree as ET

In [2]:
import numpy as np
import cv2
from shapely.geometry import Polygon

def mask_to_polygons( mask):
    '''
    Convierte una máscara de imagen en polígonos. Devuelve dos listas:
    - Lista de polígonos de shapely sin normalizar
    - Lista de polígonos de shapely normalizados (coordenadas entre 0 y 1)

    Args:
        img_path (str): Ruta al archivo de imagen original.
        mask_path (str): Ruta al archivo de la máscara en escala de grises.
    '''
    
    # mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Calcula los contornos 
    mask = mask.astype(bool)
    #contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # convertimos los contornos a polígonos de Label Studio
    polygons = []
    normalized_polygons = []
    for contour in contours:
        
        # Lo meto en un try porque la extraccion de polígonos que hace el opencv a partir de la máscara
        # a veces genera polígonos de menos de 4 vértices, que no tiene sentido por no ser cerrados, 
        # provocando que falle al convertir a polígno de shapely

        try:
            polygon = contour.reshape(-1, 2).tolist()
          
            # normalizamos las coordenadas entre 0 y 1 porque así lo requiere YOLOv8
            normalized_polygon = [[round(coord[0] / mask.shape[1] , 4), round(coord[1] / mask.shape[0] , 4)] for coord in polygon]
        
            # Convertimos a objeto poligono de shapely (sin normalizar)
            polygon_shapely = Polygon(polygon)
            simplified_polygon = polygon_shapely.simplify(0.85, preserve_topology=True)
            polygons.append(simplified_polygon)

            # normalizdos
            normalized_polygons.append(Polygon(normalized_polygon))
          

        except Exception as e:
            pass
        

    return normalized_polygons 

In [10]:
mask_path = "/mnt/DATA/dronetracking/Abhin/segement&detect/U2net/video1/osm_0001.png"
mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

In [11]:
normalized_polygons = mask_to_polygons(mask)

In [12]:
normalized_polygons

[<POLYGON ((0.585 0.516, 0.584 0.518, 0.584 0.518, 0.584 0.519, 0.584 0.53, 0...>]

In [3]:
def process_masks_in_folder(mask_folder, output_folder):
    '''
    Procesa todas las máscaras en una carpeta y crea un archivo de texto para cada imagen con los polígonos.
    
    Args:
        mask_folder (str): Ruta a la carpeta de máscaras.
        output_folder (str): Ruta a la carpeta donde se guardarán los archivos de texto.
    '''
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for mask_filename in os.listdir(mask_folder):
        if not mask_filename.endswith('.png'):
            continue

        mask_path = os.path.join(mask_folder, mask_filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        polygons = mask_to_polygons(mask)
        
        # Assuming the output filename should follow the pattern without the '_original_size_mask' suffix
        output_filename = mask_filename.replace('_original_size_mask', '').replace('.png', '.txt')
        output_path = os.path.join(output_folder, output_filename)
        
        with open(output_path, 'w') as f:
            for poly in polygons:
                f.write('0 ' + ' '.join(f"{(coord[0])} {(coord[1])}" for coord in poly.exterior.coords[:-1]) + '\n')

In [5]:
# Rutas a las carpetas
mask_folder = '/mnt/DATA/dronetracking/Abhin/segement_detect/drone_detection_icpr_new/train/original_sized_mask'
output_folder = '/mnt/DATA/dronetracking/Abhin/segement_detect/drone_detection_icpr_new/train/norm_polygons'

process_masks_in_folder(mask_folder, output_folder)